Import modules

In [1]:
import numpy as np
import functools
import labkey
import pandas as pd

Helper function 

In [2]:
def labkey_to_df(sql_query, database, maxrows):
  """generate an pandas dataframe from labkey sql query

  Args:
    sql_query (str): an sql query as string.
    dr (str): GEL datarelease version
    """
  ver = labkey.__version__

  if ver == '1.2.0' or ver == '1.4.0' or ver == '1.4.1':
    server_context = labkey.utils.create_server_context(
      domain= "labkey-embassy.gel.zone",
      container_path = database,
      context_path = "labkey",
      use_ssl = True
    )

    results =  labkey.query.execute_sql(
      server_context,
      schema_name="lists",
      sql=sql_query,
      max_rows=maxrows
    )

  elif ver == '2.4.0':
    from labkey.api_wrapper import APIWrapper

    labkey_server = "labkey-embassy.gel.zone"
    project_name = database  # Project folder name
    contextPath = "labkey"
    schema = 'lists'
    api = APIWrapper(
    labkey_server, 
    project_name, 
      contextPath, 
      use_ssl=True)

    results = api.query.execute_sql(sql=sql_query, schema_name=schema, max_rows = maxrows)

  return(pd.DataFrame(results['rows']))

100kGP

In [3]:
version = "/main-programme/main-programme_v18_2023-12-21"

Case cohort

In [4]:
disease = "INSERT DISEASE"

recruited_sql = (f'''
    SELECT participant_id, 
        normalised_specific_disease, 
        normalised_disease_sub_group, 
        normalised_disease_group
    FROM rare_diseases_participant_disease
    WHERE normalised_specific_disease = '{disease}'
    ''')

In [5]:
recruited_query = labkey_to_df(recruited_sql, version, 10000)
recruited_query

,normalised_disease_group,participant_id,normalised_specific_disease,normalised_disease_sub_group
0,Skeletal disorders,111001823,Unexplained skeletal dysplasia,Skeletal dysplasias
1,Skeletal disorders,111002145,Unexplained skeletal dysplasia,Skeletal dysplasias
2,Skeletal disorders,111002234,Unexplained skeletal dysplasia,Skeletal dysplasias
3,Skeletal disorders,111002239,Unexplained skeletal dysplasia,Skeletal dysplasias
4,Skeletal disorders,111002343,Unexplained skeletal dysplasia,Skeletal dysplasias
...,...,...,...,...
302,Skeletal disorders,210018992,Unexplained skeletal dysplasia,Skeletal dysplasias
303,Skeletal disorders,210019353,Unexplained skeletal dysplasia,Skeletal dysplasias
304,Skeletal disorders,210019489,Unexplained skeletal dysplasia,Skeletal dysplasias
305,Skeletal disorders,210019619,Unexplained skeletal dysplasia,Skeletal dysplasias


In [6]:
recruited_participants = recruited_query['participant_id'].tolist()

In [7]:
participants = list(dict.fromkeys(recruited_participants))
len(participants)

307

In [8]:
solved = "yes"

gmc_sql = (f'''
    SELECT participant_id, case_solved_family
    FROM gmc_exit_questionnaire
    WHERE participant_id in {*participants,}
    AND case_solved_family != '{solved}'
    ''')

gmc_query = labkey_to_df(gmc_sql, version, 1000)
gmc_query

,participant_id,case_solved_family
0,111001823,no
1,111002145,no
2,111002234,no
3,111002343,no
4,111002675,no
...,...,...
219,210018735,no
220,210018975,no
221,210018992,no
222,210019353,no


In [9]:
dd_sql = (f'''
    SELECT participant_id
    FROM submitted_diagnostic_discovery
    WHERE participant_id IN {*gmc_query['participant_id'].tolist(),}
    ''')

dd_query = labkey_to_df(dd_sql, version, 1000)
dd_query

,participant_id
0,111003462
1,112003736
2,112003736
3,112005591
4,112006466
5,114001869
6,114001869
7,115011772
8,115011772
9,115011924


In [10]:
cohort = [i for i in gmc_query['participant_id'].tolist() if i not in dd_query['participant_id'].tolist()]
len(cohort)

208

ICD 10 cases

Control cohort

In [11]:
not_disease = "Skeletal disorders"

recruited_not_sql = (f'''
    SELECT participant_id, 
        normalised_specific_disease, 
        normalised_disease_sub_group, 
        normalised_disease_group
    FROM rare_diseases_participant_disease
    WHERE normalised_disease_group = '{not_disease}'
    ''')

recruited_not_query = labkey_to_df(recruited_not_sql, version, 100000)
recruited_not_list = recruited_not_query['participant_id'].tolist()
len(recruited_not_list)

1095

In [12]:
icd_not_codes = ['Q30.0','Q77.3','Q75.0','78.3','Q78.0']

hes_tables = ["apc", "op", "ae"]

icd_not_concat = pd.DataFrame()

diag_not_statement = "%' OR diag_all LIKE '%".join(icd_not_codes)

for hes_table in hes_tables:
    sqlstr = (
        f'''
        SELECT participant_id, diag_all
        FROM hes_{hes_table}
        WHERE diag_all LIKE '%{diag_not_statement}%'
        ''')
    icd_query = labkey_to_df(sqlstr, version, 1000000)
    if not icd_query.empty:
        icd_not_concat = pd.concat([icd_query, icd_not_concat])

icd_not_list = icd_not_concat['participant_id'].tolist()
len(icd_not_list)

2

In [13]:
not_list = recruited_not_list + icd_not_list 
not_list = list(dict.fromkeys(not_list))
len(not_list)

1095

In [14]:
participant_sql = 'SELECT participant_id FROM participant'
participant_list = labkey_to_df(participant_sql, version, 1000000)['participant_id'].tolist()
control = [i for i in participant_list if i not in not_list]
len(control)

89083

Match demographics

In [15]:
from statistics import mean

case_sql = (f'''
    SELECT participant_id, yob, 
    participant_phenotyped_sex, participant_karyotyped_sex
    FROM participant_summary
    WHERE participant_id IN {*cohort,}
    ''')
case_query = labkey_to_df(case_sql, version, 10000).drop_duplicates()
case_yob_mean = mean(case_query['yob'].tolist())

control_sql = (f'''
    SELECT participant_id, yob, 
    participant_phenotyped_sex, participant_karyotyped_sex
    FROM participant_summary
    WHERE participant_id IN {*control,}
    ''')
control_query = labkey_to_df(control_sql, version, 100000).drop_duplicates()
control_yob_mean = mean(control_query['yob'].tolist())

print("case mean = ", case_yob_mean, 
      "\ncontrol mean = ", control_yob_mean)

case mean =  1998.4619565217392 
control mean =  1980.59345420891


In [16]:
print("case XX = ", case_query['participant_karyotyped_sex'].value_counts()['XX']/len(case_query),
     "\ncase Female = ", case_query['participant_phenotyped_sex'].value_counts()['Female']/len(case_query),
     "\ncase XY = ", case_query['participant_karyotyped_sex'].value_counts()['XY']/len(case_query),
     "\ncase Male = ", case_query['participant_phenotyped_sex'].value_counts()['Male']/len(case_query),
     "\ncontrol XX = ", control_query['participant_karyotyped_sex'].value_counts()['XX']/len(control_query),
     "\ncontrol Female = ", control_query['participant_phenotyped_sex'].value_counts()['Female']/len(control_query),
     "\ncontrol XY = ", control_query['participant_karyotyped_sex'].value_counts()['XY']/len(control_query),
     "\ncontrol Male = ", control_query['participant_phenotyped_sex'].value_counts()['Male']/len(control_query))

case XX =  0.5163043478260869 
case Female =  0.5217391304347826 
case XY =  0.4673913043478261 
case Male =  0.4782608695652174 
control XX =  0.41655799082916156 
control Female =  0.5327079214168524 
control XY =  0.36485633755416264 
control Male =  0.46729207858314753


In [17]:
eth = "European"

case_ethnicity_sql = (f'''
    SELECT participant_id, genetically_inferred_ancestry_thr
    FROM participant_summary
    WHERE genetically_inferred_ancestry_thr = '{eth}'
    AND participant_id IN {*cohort,}
    ''')
case_ethnicity_query = labkey_to_df(case_ethnicity_sql, version, 10000).drop_duplicates()
filtered_case = case_ethnicity_query['participant_id'].tolist()

control_ethnicity_sql = (f'''
    SELECT participant_id, genetically_inferred_ancestry_thr
    FROM participant_summary
    WHERE genetically_inferred_ancestry_thr = '{eth}'
    AND participant_id IN {*control,}
    ''')
control_ethnicity_query = labkey_to_df(control_ethnicity_sql, version, 100000).drop_duplicates()
filtered_control = control_ethnicity_query['participant_id'].tolist()

General inclusion critera

In [18]:
case_df = pd.DataFrame(filtered_case, columns = ['participant_id'])
case_df['case'] = "case"
control_df = pd.DataFrame(filtered_control, columns = ['participant_id'])
control_df['case'] = "control"

case_control_table = pd.concat([case_df, control_df]).drop_duplicates()
case_control_table

,participant_id,case
0,111001823,case
1,111002145,case
2,111002234,case
3,111002675,case
4,111002677,case
...,...,...
23596,115018231,control
23597,115018232,control
23598,115018235,control
23599,115018236,control


In [19]:
sex_sql = (f'''
    SELECT participant_id, participant_phenotyped_sex, participant_karyotyped_sex
    FROM participant_summary
    WHERE participant_id IN {*case_control_table['participant_id'].tolist(),} 
    AND (participant_karyotyped_sex = 'XX'
    OR participant_karyotyped_sex = 'XY')
    ''')
sex_query = labkey_to_df(sex_sql, version, 100000).drop_duplicates()

sex_query = sex_query.drop(sex_query[ (sex_query['participant_phenotyped_sex'] == 'Male') & (sex_query['participant_karyotyped_sex'] == 'XX') ].index)
sex_query = sex_query.drop(sex_query[ (sex_query['participant_phenotyped_sex'] == 'Female') & (sex_query['participant_karyotyped_sex'] == 'XY') ].index)

case_control_table = case_control_table.loc[
    case_control_table['participant_id'].isin
    (sex_query['participant_id'].tolist())]
case_control_table

,participant_id,case
0,111001823,case
1,111002145,case
2,111002234,case
3,111002675,case
4,111002677,case
...,...,...
23596,115018231,control
23597,115018232,control
23598,115018235,control
23599,115018236,control


In [20]:
aggregate_sql = (f'''
    SELECT participant_id, platekey
    FROM aggregate_gvcf_sample_stats
    WHERE participant_id IN {*case_control_table['participant_id'].tolist(),}
    AND sample_source = 'BLOOD'
    AND sample_preparation_method = 'EDTA'
    AND sample_library_type = 'TruSeq PCR-Free High Throughput'
    ''')
aggregate_query = labkey_to_df(aggregate_sql, version, 100000)
case_control_table = aggregate_query.merge(case_control_table).drop_duplicates()
case_control_table

,platekey,participant_id,case
0,LP3000448-DNA_A09,111000003,control
1,LP2000950-DNA_C02,111000004,control
2,LP2000913-DNA_A02,111000005,control
3,LP3000096-DNA_A06,111000017,control
4,LP3000085-DNA_G07,111000018,control
...,...,...,...
23228,LP3000547-DNA_B04,210015497,case
23229,LP3000753-DNA_G11,210016039,case
23230,LP3000849-DNA_F06,210016214,case
23231,LP3001034-DNA_A08,210018308,case


In [21]:
relatedness_table = pd.read_csv('/gel_data_resources/main_programme/aggregation/aggregate_gVCF_strelka/aggV2/additional_data/PCs_relatedness/relatedness/GEL_aggV2_MAF5_mp10_0.0442.kin0',sep = '\t')
relatedness_table

,#FID1,IID1,FID2,IID2,NSNP,HETHET,IBS0,KINSHIP
0,125000010_E046339_2007_E000097016,125000010_E046339_2007_E000097016,125000007_E068978_1965_E000086324,125000007_E068978_1965_E000086324,63523,0.183319,0.000031,0.248567
1,125000010_E046339_2007_E000097016,125000010_E046339_2007_E000097016,125000008_E047442_1980_E000086325,125000008_E047442_1980_E000086325,63523,0.184783,0.000047,0.250513
2,125000013_G292573_1977_G003161041,125000013_G292573_1977_G003161041,125000011_G262141_2007_G002156515,125000011_G262141_2007_G002156515,63523,0.183193,0.000094,0.247087
3,125000014_G292572_1975_G003161042,125000014_G292572_1975_G003161042,125000011_G262141_2007_G002156515,125000011_G262141_2007_G002156515,63523,0.185256,0.000094,0.247077
4,125000016_G324422_1961_G003172133,125000016_G324422_1961_G003172133,125000015_G285963_2002_G002168862,125000015_G285963_2002_G002168862,63523,0.190073,0.000031,0.253805
...,...,...,...,...,...,...,...,...
144970,LP3001752-DNA_B01,LP3001752-DNA_B01,LP3001604-DNA_G01,LP3001604-DNA_G01,63523,0.189160,0.071423,0.052147
144971,LP3001752-DNA_B01,LP3001752-DNA_B01,LP3001636-DNA_F07,LP3001636-DNA_F07,63523,0.188483,0.071533,0.052264
144972,LP3001752-DNA_D01,LP3001752-DNA_D01,LP3000014-DNA_A02,LP3000014-DNA_A02,63523,0.187271,0.000016,0.250454
144973,LP3001752-DNA_D01,LP3001752-DNA_D01,LP3000448-DNA_C08,LP3000448-DNA_C08,63523,0.177888,0.034633,0.143915


In [22]:
mz_twins_table = relatedness_table[ (relatedness_table['KINSHIP'] > 0.354) ]
mz_twins_table

,#FID1,IID1,FID2,IID2,NSNP,HETHET,IBS0,KINSHIP
909,LP3000018-DNA_C01,LP3000018-DNA_C01,LP3000017-DNA_E01,LP3000017-DNA_E01,63523,0.371425,0.0,0.499958
1047,LP3000035-DNA_F11,LP3000035-DNA_F11,LP3000030-DNA_D07,LP3000030-DNA_D07,63523,0.372164,0.0,0.499958
1075,LP3000038-DNA_F05,LP3000038-DNA_F05,LP3000036-DNA_G03,LP3000036-DNA_G03,63523,0.374620,0.0,0.499926
1299,LP3000092-DNA_A06,LP3000092-DNA_A06,LP3000085-DNA_B07,LP3000085-DNA_B07,63523,0.340444,0.0,0.499908
1340,LP3000096-DNA_A06,LP3000096-DNA_A06,LP3000092-DNA_A07,LP3000092-DNA_A07,63523,0.374337,0.0,0.499905
...,...,...,...,...,...,...,...,...
141907,LP3001604-DNA_A02,LP3001604-DNA_A02,LP3001189-DNA_G09,LP3001189-DNA_G09,63523,0.371362,0.0,0.499672
143137,LP3001604-DNA_H04,LP3001604-DNA_H04,LP3001491-DNA_H07,LP3001491-DNA_H07,63523,0.370653,0.0,0.499724
143286,LP3001623-DNA_F10,LP3001623-DNA_F10,LP3000358-DNA_D07,LP3000358-DNA_D07,63523,0.373235,0.0,0.499736
144193,LP3001659-DNA_C09,LP3001659-DNA_C09,LP3001538-DNA_H07,LP3001538-DNA_H07,63523,0.370212,0.0,0.499851


In [23]:
twins_list = mz_twins_table[['IID1', 'IID2']].values.tolist()

case_control_list = case_control_table['platekey'].tolist()

for twins in twins_list:
    if twins[0] in case_control_list:
        if (case_control_table.query('platekey == @twins[0]')['case']).values == 'case':
            case_control_table = case_control_table.drop(case_control_table[case_control_table['platekey'] == twins[1]].index)
            print (twins[1])
    elif twins[1] in case_control_list:
        if (case_control_table.query('platekey == @twins[1]')['case']).values == 'case':
            case_control_table = case_control_table.drop(case_control_table[case_control_table['platekey'] == twins[0]].index)
            print(twins[0])
case_control_table

LP3000869-DNA_G03


,platekey,participant_id,case
0,LP3000448-DNA_A09,111000003,control
1,LP2000950-DNA_C02,111000004,control
2,LP2000913-DNA_A02,111000005,control
3,LP3000096-DNA_A06,111000017,control
4,LP3000085-DNA_G07,111000018,control
...,...,...,...
23228,LP3000547-DNA_B04,210015497,case
23229,LP3000753-DNA_G11,210016039,case
23230,LP3000849-DNA_F06,210016214,case
23231,LP3001034-DNA_A08,210018308,case


Filepaths

In [24]:
filetype = "Genomic VCF"

path_sql = (f'''
    SELECT participant_id, platekey, filename, file_path
    FROM genome_file_paths_and_types
    WHERE participant_id IN {*case_control_table['participant_id'].tolist(),}
    AND file_sub_type = '{filetype}'
''')

path_query = labkey_to_df(path_sql, version, 100000)
case_control_paths = path_query.merge(case_control_table)
case_control_paths

,file_path,filename,platekey,participant_id,case
0,/genomes/by_date/2018-01-19/HX03031897/LP30004...,LP3000448-DNA_A09.genome.vcf.gz,LP3000448-DNA_A09,111000003,control
1,/genomes/by_date/2015-12-16/0000205553/LP20009...,LP2000950-DNA_C02.genome.vcf.gz,LP2000950-DNA_C02,111000004,control
2,/genomes/by_date/2018-11-23/BE00001372/LP20009...,LP2000950-DNA_C02.genome.vcf.gz,LP2000950-DNA_C02,111000004,control
3,/genomes/by_date/2015-12-31/0000286408/LP20009...,LP2000913-DNA_A02.genome.vcf.gz,LP2000913-DNA_A02,111000005,control
4,/genomes/by_date/2018-11-21/BE00001344/LP20009...,LP2000913-DNA_A02.genome.vcf.gz,LP2000913-DNA_A02,111000005,control
...,...,...,...,...,...
24002,/genomes/by_date/2018-02-03/HX10725871/LP30005...,LP3000547-DNA_B04.genome.vcf.gz,LP3000547-DNA_B04,210015497,case
24003,/genomes/by_date/2018-06-02/HX10988170/LP30007...,LP3000753-DNA_G11.genome.vcf.gz,LP3000753-DNA_G11,210016039,case
24004,/genomes/by_date/2018-06-14/HX11618687/LP30008...,LP3000849-DNA_F06.genome.vcf.gz,LP3000849-DNA_F06,210016214,case
24005,/genomes/by_date/2018-10-30/HX12731626/LP30010...,LP3001034-DNA_A08.genome.vcf.gz,LP3001034-DNA_A08,210018308,case


In [25]:
count_control = (case_control_paths['case'] == 'control').sum()
count_case = (case_control_paths['case'] == 'case').sum()
print(f'case - {count_case}')
print(f'control - {count_control}')

case - 125
control - 23882


In [27]:
case_paths = case_control_paths[case_control_paths['case'] == 'case']
control_paths = case_control_paths[case_control_paths['case'] == 'control']
case_paths.to_csv('Part1_case_paths_output.csv', index=False)
control_paths.to_csv('Part1_control_paths_output.csv', index=False)
case_sample_size = pd.DataFrame([count_case])
case_sample_size.to_csv('case_sample_size.csv', index=False, header=False)